In [1]:
import os
import numpy as np
import cv2
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
directory = "TImages/"

In [3]:
image_size = (244, 244)
classes = ["0", "1", "2"]

In [4]:
# Create an ImageDataGenerator instance with data augmentation settings
datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest',
    samplewise_center=True,
    samplewise_std_normalization=True,
)

In [5]:
training_data = []

def create_training_data():
    for category in classes:
        path = os.path.join(directory, category)
        class_num = classes.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img))
                new_array = cv2.resize(img_array, image_size)

                # Generate and store augmented images
                augmented_images = []
                augmented_images.append(new_array)  # Original image
                img_array_aug = new_array.reshape((1,) + new_array.shape)
                i = 0
                for batch in datagen.flow(img_array_aug, batch_size=1):
                    augmented_images.append(batch[0])
                    i += 1
                    if i >= 5:  # Generate 5 augmented images per input image
                        break

                for augmented_image in augmented_images:
                    image_hsv = cv2.cvtColor(augmented_image, cv2.COLOR_BGR2HSV)
                    training_data.append([image_hsv, class_num])

            except Exception as e:
                pass

In [6]:
create_training_data()

In [7]:
lenofimage = len(training_data)
print(lenofimage)

5850


In [8]:
# Shuffle the training data to ensure randomness
np.random.shuffle(training_data)

In [9]:
# Separate images (X) and labels (y)
X = []
y = []

for features, label in training_data:
    X.append(features)
    y.append(label)

# Convert lists to numpy arrays
X = np.array(X)
y = np.array(y)

In [10]:
X = X / 255.0

In [11]:
# Print the shapes to verify
print("Shape of X:", X.shape)
print("Shape of y:", y.shape)

Shape of X: (5850, 244, 244, 3)
Shape of y: (5850,)


In [12]:
from sklearn.model_selection import train_test_split

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [15]:
# Define the CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(244, 244, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(classes), activation='softmax'))

In [16]:
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [17]:
# Train the model
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10
147/147 [==============================] - 188s 1s/step - loss: 0.3201 - accuracy: 0.9308 - val_loss: 0.2241 - val_accuracy: 0.9291
Epoch 2/10
147/147 [==============================] - 196s 1s/step - loss: 0.1927 - accuracy: 0.9368 - val_loss: 0.3004 - val_accuracy: 0.9325
Epoch 3/10
147/147 [==============================] - 216s 1s/step - loss: 0.1876 - accuracy: 0.9438 - val_loss: 0.2226 - val_accuracy: 0.9530
Epoch 4/10
147/147 [==============================] - 221s 2s/step - loss: 0.1670 - accuracy: 0.9509 - val_loss: 0.2200 - val_accuracy: 0.9538
Epoch 5/10
147/147 [==============================] - 218s 1s/step - loss: 0.1665 - accuracy: 0.9521 - val_loss: 0.2891 - val_accuracy: 0.9581
Epoch 6/10
147/147 [==============================] - 220s 1s/step - loss: 0.1706 - accuracy: 0.9562 - val_loss: 0.4002 - val_accuracy: 0.9581
Epoch 7/10
147/147 [==============================] - 217s 1s/step - loss: 0.1483 - accuracy: 0.9573 - val_loss: 0.6158 - val_accuracy: 0.9556

In [18]:
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 242, 242, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 121, 121, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 119, 119, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 59, 59, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 57, 57, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 28, 28, 128)       0

In [19]:
from tensorflow.keras.models import Model

# Create a new model that extracts features from the last convolutional layer
feature_extraction_model = Model(inputs=model.input, outputs=model.get_layer('flatten').output)

# Extract features from your dataset
X_train_features = feature_extraction_model.predict(X_train)
X_test_features = feature_extraction_model.predict(X_test)


37/37 [==============================] - 7s 174ms/step


In [20]:
# from sklearn.svm import SVC

# # Create and train an SVM classifier
# svm_classifier = SVC(kernel='linear', C=1.0, verbose=2)
# svm_classifier.fit(X_train_features, y_train)

# # Evaluate the SVM classifier on the validation set
# svm_accuracy = svm_classifier.score(X_test_features, y_test)
# print("SVM Validation Accuracy:", svm_accuracy)


In [21]:
## for NAIVE BAYES

In [22]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [23]:
# Feature Scaling
sc = StandardScaler()

X_train_nb = sc.fit_transform(X_train_features)
X_test_nb = sc.transform(X_test_features)

In [24]:
# Define the parameter grid for hyperparameter tuning
param_grid = {
    'var_smoothing': [1e-9, 1e-5] # You can adjust this range
    
}

In [25]:
nb = GaussianNB()

In [26]:
# Create the GridSearch object without cross-validation
grid_search = GridSearchCV(nb, param_grid, cv=5, verbose=2)  # Set cv=None for no cross-validation

In [27]:
grid_search.class_prior_ = [0.5, 0.5]

In [28]:
# Fit the GridSearch object to your data
grid_search.fit(X_train_features, y_train)


Fitting 5 folds for each of 2 candidates, totalling 10 fits
[CV] END ................................var_smoothing=1e-09; total time=   6.2s
[CV] END ................................var_smoothing=1e-09; total time=   5.0s
[CV] END ................................var_smoothing=1e-09; total time=   4.8s
[CV] END ................................var_smoothing=1e-09; total time=   4.8s
[CV] END ................................var_smoothing=1e-09; total time=   4.8s
[CV] END ................................var_smoothing=1e-05; total time=   4.8s
[CV] END ................................var_smoothing=1e-05; total time=   4.8s
[CV] END ................................var_smoothing=1e-05; total time=   4.8s
[CV] END ................................var_smoothing=1e-05; total time=   4.8s
[CV] END ................................var_smoothing=1e-05; total time=   4.8s


GridSearchCV(cv=5, estimator=GaussianNB(),
             param_grid={'var_smoothing': [1e-09, 1e-05]}, verbose=2)

In [29]:
# Get the best estimator and evaluate it
best_nb = grid_search.best_estimator_
y_test_pred_nb = best_nb.predict(X_test_features)
accuracy_nb = accuracy_score(y_test, y_test_pred_nb)
print("Best Gaussian Naive Bayes Parameters:", grid_search.best_params_)
print("Accuracy: {:.2f}%".format(accuracy_nb * 100))

Best Gaussian Naive Bayes Parameters: {'var_smoothing': 1e-09}
Accuracy: 43.25%


In [30]:
## for SVM

In [31]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [32]:
# Feature Scaling
sc = StandardScaler()

X_train_svm = sc.fit_transform(X_train_features)
X_test_svm = sc.transform(X_test_features)

In [33]:
# Hyperparameter Tuning using GridSearchCV
param_grid_svm = {'C': [5], 'gamma': ['auto'], 'kernel': ['rbf']}

In [34]:
# Train the SVM model
svm = SVC()
grid_search = GridSearchCV(svm, param_grid_svm, cv=5, verbose=2)
grid_search.fit(X_train_svm, y_train)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END ........................C=5, gamma=auto, kernel=rbf; total time= 4.3min
[CV] END ........................C=5, gamma=auto, kernel=rbf; total time= 5.2min
[CV] END ........................C=5, gamma=auto, kernel=rbf; total time= 6.1min
[CV] END ........................C=5, gamma=auto, kernel=rbf; total time= 5.5min
[CV] END ........................C=5, gamma=auto, kernel=rbf; total time= 5.5min


GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': [5], 'gamma': ['auto'], 'kernel': ['rbf']},
             verbose=2)

In [35]:
# # Get the best estimator and evaluate it
best_svm = grid_search.best_estimator_
y_test_pred_svm = best_svm.predict(X_test_svm)
accuracy_svm = accuracy_score(y_test, y_test_pred_svm)
print("Best SVM Parameters:", grid_search.best_params_)
print("Accuracy: {:.2f}%".format(accuracy_svm * 100))

Best SVM Parameters: {'C': 5, 'gamma': 'auto', 'kernel': 'rbf'}
Accuracy: 95.90%
